#### Este dataset contém imagens de campos de trigo, com caixas delimitadoras para cada espiga de trigo identificada. Algumas imagens podem não conter espigas de trigo ou caixas delimitadoras. As imagens foram capturadas em vários locais ao redor do mundo.

```
Formato dos Dados:
Imagens: Arquivos de imagens (.jpg) contendo campos de trigo, com ou sem espigas visíveis.
Caixas Delimitadoras: Cada espiga de trigo identificada tem uma caixa delimitadora associada, armazenada em um arquivo CSV.
Arquivo CSV: O arquivo train.csv contém:
image_id: ID da imagem (corresponde ao nome do arquivo da imagem)
width e height: Largura e altura da imagem
bbox: Coordenadas da caixa delimitadora para cada espiga de trigo. Há uma linha no CSV para cada caixa delimitadora.
Nem todas as imagens têm caixas delimitadoras.
```

Baixar o dataset:https://www.kaggle.com/competitions/global-wheat-detection/data

Instalação de Pacotes

In [ ]:
%pip install pandas
%pip install tqdm

Convert Global Wheat Detection DS the train.csv to per image labelme json files

In [5]:

import pandas as pd
from IPython.display import Image
import json
import os
from tqdm import tqdm

In [ ]:
d = pd.read_csv('./dataset/global-wheat-detection/train.csv')
d['image_id'].unique()[0]
for image_id in tqdm(list(d['image_id'].unique())):
    shapes = []
    for _, row in d[d['image_id']==image_id].iterrows():
        x, y, w, h = json.loads(row['bbox'])
        shapes.append({
          "label": "wheat",
          "points": [[x, y], [x+w, y+h]],
          "group_id": None,
          "shape_type": "rectangle",
          "flags": {}
        })
    img_filename = "{}.jpg".format(image_id)
    labelme = json.dumps({
      "version": "4.0.0",
      "flags": {},
      "shapes": shapes,
      "imagePath": img_filename,
      "imageData": None,
      "imageHeight": 1024,
      "imageWidth": 1024
    }, indent=2)
    annot_filename = "labelme/{}.json".format(image_id)
    os.makedirs(os.path.dirname(annot_filename), exist_ok=True)
    with open(annot_filename, "w") as f:
        f.write(labelme)

In [ ]:
import os
import shutil

# Caminhos das pastas
dataset_path = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection'
labels_path = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\labelme'

# Listar todos os arquivos .json na pasta de labels
label_files = [f for f in os.listdir(labels_path) if f.endswith('.json')]

# Mover cada arquivo .json para a pasta correspondente
for label_file in label_files:
    # Nome do arquivo sem extensão
    file_name = os.path.splitext(label_file)[0]
    
    # Caminho completo do arquivo de label
    src_path = os.path.join(labels_path, label_file)
    
    # Verificar se o arquivo de imagem correspondente está na pasta train ou test
    if os.path.exists(os.path.join(dataset_path, 'train', f'{file_name}.jpg')):
        dest_path = os.path.join(dataset_path, 'train', label_file)
    elif os.path.exists(os.path.join(dataset_path, 'test', f'{file_name}.jpg')):
        dest_path = os.path.join(dataset_path, 'test', label_file)
    else:
        print(f'Imagem correspondente para {label_file} não encontrada em train ou test.')
        continue
    
    # Mover o arquivo de label para a pasta correspondente
    shutil.move(src_path, dest_path)
    print(f'Movido {label_file} para {dest_path}')